<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import pandas as pd
from collections import defaultdict

def run_multi_group_simulation(projects, groups, points_per_month):
    """
    Optimizes monthly point allocation for multiple groups and shared projects.

    Args:
        projects (list): A list of project dictionaries, each with 'name', 'points_needed',
                         and 'deadline_months'.
        groups (dict): A dictionary where keys are group names and values are lists
                       of project names associated with that group.
        points_per_month (int): The number of points each group can distribute monthly.

    Returns:
        pandas.DataFrame: A DataFrame showing the points allocated by each group
                          to each project each month.
    """
    # Initialize projects
    current_projects = {p['name']: p.copy() for p in projects}
    for name in current_projects:
        current_projects[name]['points_remaining'] = current_projects[name]['points_needed']

    month = 1
    allocation_history = []
    
    print("--- Multi-Group Project Allocation Simulation Starting ---")

    while any(p['points_remaining'] > 0 for p in current_projects.values()):
        print(f"\n--- Month {month} ---")
        
        # Track available points for each group this month
        group_points_available = {group: points_per_month for group in groups}
        
        # Identify active projects and calculate global urgency
        active_projects = [p for p in current_projects.values() if p['points_remaining'] > 0]
        
        for proj in active_projects:
            months_left = proj['deadline_months'] - (month - 1)
            if months_left <= 0:
                proj['urgency'] = float('inf')
                print(f"🚨 URGENT: Project '{proj['name']}' is at or past its deadline!")
            else:
                proj['urgency'] = proj['points_remaining'] / months_left
        
        # Sort projects by global urgency
        active_projects.sort(key=lambda p: p['urgency'], reverse=True)
        print(f"Global Project Priority: {[p['name'] for p in active_projects]}")

        # Allocate points based on sorted priority
        for proj in active_projects:
            if proj['points_remaining'] <= 0:
                continue

            # Find groups associated with this project that still have points
            participating_groups = [g for g in groups if proj['name'] in groups[g] and group_points_available[g] > 0]
            
            if not participating_groups:
                continue

            points_to_allocate_to_proj = proj['points_remaining']
            
            # Distribute the need for points among participating groups
            for group in participating_groups:
                if points_to_allocate_to_proj <= 0:
                    break

                points_from_group = min(group_points_available[group], points_to_allocate_to_proj)
                
                group_points_available[group] -= points_from_group
                proj['points_remaining'] -= points_from_group
                points_to_allocate_to_proj -= points_from_group
                
                # Record this specific allocation
                allocation_history.append({
                    'Month': f"Month {month}",
                    'Group': group,
                    'Project': proj['name'],
                    'Points Allocated': points_from_group
                })
                print(f"Group '{group}' allocated {points_from_group} points to '{proj['name']}'.")

            if proj['points_remaining'] == 0:
                print(f"🎉 Project '{proj['name']}' completed in Month {month}! 🎉")

        month += 1
        # Safety break to prevent infinite loops in case of impossible scenarios
        if month > 100:
            print("Simulation stopped to prevent an infinite loop.")
            break
            
    print("\n--- Simulation Complete ---")
    
    if not allocation_history:
        return pd.DataFrame()

    # Create and pivot the DataFrame for a clear report
    df = pd.DataFrame(allocation_history)
    pivot_df = df.pivot_table(index=['Project', 'Group'], columns='Month', values='Points Allocated', fill_value=0)
    
    return pivot_df

if __name__ == "__main__":
    # --- Your Project and Group Data ---
    my_projects = [
        {'name': 'Project Alpha', 'points_needed': 30, 'deadline_months': 3},
        {'name': 'Project Beta',  'points_needed': 40, 'deadline_months': 4},
        {'name': 'Project Gamma', 'points_needed': 25, 'deadline_months': 5},
    ]

    # Define which groups are working on which projects
    my_groups = {
        'Group 1': ['Project Alpha', 'Project Beta'],
        'Group 2': ['Project Beta', 'Project Gamma'],
    }

    MONTHLY_POINTS_PER_GROUP = 10

    # Run the simulation
    allocation_plan_df = run_multi_group_simulation(my_projects, my_groups, MONTHLY_POINTS_PER_GROUP)
    
    print("\n--- Final Allocation Plan ---")
    print(allocation_plan_df)

--- Multi-Group Project Allocation Simulation Starting ---

--- Month 1 ---
Global Project Priority: ['Project Alpha', 'Project Beta', 'Project Gamma']
Group 'Group 1' allocated 10 points to 'Project Alpha'.
Group 'Group 2' allocated 10 points to 'Project Beta'.

--- Month 2 ---
Global Project Priority: ['Project Alpha', 'Project Beta', 'Project Gamma']
Group 'Group 1' allocated 10 points to 'Project Alpha'.
Group 'Group 2' allocated 10 points to 'Project Beta'.

--- Month 3 ---
Global Project Priority: ['Project Alpha', 'Project Beta', 'Project Gamma']
Group 'Group 1' allocated 10 points to 'Project Alpha'.
🎉 Project 'Project Alpha' completed in Month 3! 🎉
Group 'Group 2' allocated 10 points to 'Project Beta'.

--- Month 4 ---
Global Project Priority: ['Project Gamma', 'Project Beta']
Group 'Group 2' allocated 10 points to 'Project Gamma'.
Group 'Group 1' allocated 10 points to 'Project Beta'.
🎉 Project 'Project Beta' completed in Month 4! 🎉

--- Month 5 ---
Global Project Priority: [

C:\Users\User\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\User\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
